In [1]:
import matplotlib
import serial
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import threading
import queue
import time
import matplotlib.animation as animation

In [2]:
SERIAL_PORT = '/dev/tty.usbserial-0001'     # Change this based on your ESP32 port
BAUD_RATE = 250000       # Match the ESP32 code
ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.05)

matplotlib.use('TkAgg')


cmap = mcolors.LinearSegmentedColormap.from_list("custom", ["red", "yellow", "green"])  # Create a red-yellow-green gradient color map
fig, ax = plt.subplots()                   # Create the plot figure and axes
matrix = np.zeros((5, 3))                  # Initialize an empty 4x4 matrix
heatmap = ax.imshow(matrix, cmap=cmap, vmin=0, vmax=3.3)  # Plot the heatmap with defined colormap and voltage range
cbar = plt.colorbar(heatmap, label="Voltage (V)")         # Add a colorbar legend to the side

ax.set_xticks(np.arange(3))                             # Set X-axis ticks from 0 to 3
ax.set_yticks(np.arange(5))                             # Set Y-axis ticks from 0 to 3
ax.set_xticklabels(["CH1", "CH2", "CH3"])        # Label columns as CH1–CH4
ax.set_yticklabels(["CH1", "CH2", "CH3", "CH4", "CH5"])        # Label rows as CH1–CH4
plt.title("5x3 Matrix Voltage Visualization")           # Add a title to the plot

cell_texts = [[ax.text(j, i, "", ha="center", va="center", color="black", fontsize=12)
               for j in range(3)] for i in range(5)]

data_queue = queue.Queue()

def update_plot(frame):                               # This function runs every 50 ms from the animation timer
    if not data_queue.empty():                        # If new matrix data is available
        new_matrix = data_queue.get()                 # Retrieve the newest matrix from the queue
        heatmap.set_data(new_matrix)                  # Update heatmap color data

        for i in range(5):                            # Loop over rows
            for j in range(3):                        # Loop over columns
                cell_texts[i][j].set_text(f"{new_matrix[i][j]:.2f}")  # Update cell text with 2 decimal places
                print(cell_texts[i][j])
    return [heatmap] + [text for row in cell_texts for text in row]

def read_arduino_data():                                           # Define a background function for continuous serial reading
    while True:                                                    # Infinite loop
        try:
            line = ser.readline().decode('utf-8', errors='ignore').strip()  # Read a line from serial, decode it
            print(f"Serial line: {line}")
            if "Matrix updated:" in line:                          # If the trigger line is received
                new_matrix = np.zeros((5, 3))                      # Initialize new matrix to fill

                for i in range(5):                                 # Expect 4 lines of row data
                    row_data = ser.readline().decode('utf-8', errors='ignore').strip().split(",")  # Read and split by comma
                    print(row_data)
                    if len(row_data) == 3:                         # Ensure exactly 4 values per row
                        new_matrix[i] = [float(val) for val in row_data]  # Convert strings to floats and store

                data_queue.put(new_matrix)                         # Push the matrix into the data queue
        except Exception as e:                                     # If there's an error, print it
            print(f"Serial read error: {e}")                       # Print error to console

data_thread = threading.Thread(target=read_arduino_data, daemon=True)  # Create a daemon thread that runs serial reading
data_thread.start()                                                    # Start the thread

ani = animation.FuncAnimation(fig, update_plot, interval=50, blit=True)  # Start the animation; update every 50ms (20Hz)
plt.show()                                                     # Show the matplotlib GUI window


Serial line: Matrix updated:
['2.2083', '2.0273', '2.2924']
['1.9217', '1.8116', '1.9741']
['2.1726', '2.1288', '2.2478']
['2.2357', '2.2927', '2.4090']
['2.1745', '2.2360', '2.4001']
Serial line: Matrix updated:
['2.2078', '2.0265', '2.2900']
['1.9209', '1.8092', '1.9727']
['2.1726', '2.1285', '2.2486']
['2.2352', '2.2895', '2.4068']
['2.1761', '2.2357', '2.3982']
Serial line: Matrix updated:
['2.2102', '2.0297', '2.2929']
['1.9220', '1.8100', '1.9727']
['2.1742', '2.1277', '2.2467']
['2.2333', '2.2897', '2.4063']
['2.1747', '2.2330', '2.3977']
Serial line: Matrix updated:
['2.2094', '2.0313', '2.2897']
['1.9212', '1.8116', '1.9727']
['2.1742', '2.1294', '2.2470']
['2.2365', '2.2913', '2.4052']
['2.1750', '2.2352', '2.3982']
Serial line: Matrix updated:
['2.2110', '2.0329', '2.2897']
['1.9212', '1.8097', '1.9722']
['2.1737', '2.1291', '2.2446']
['2.2341', '2.2913', '2.4036']
['2.1756', '2.2363', '2.3977']
Text(0, 0, '2.21')
Text(1, 0, '2.03')
Text(2, 0, '2.29')
Text(0, 1, '1.92')
Text

/var/folders/b5/cbnlm0vn1j54xy3c75jt4gwc0000gn/T/ipykernel_22244/490885734.py:57: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, update_plot, interval=50, blit=True)  # Start the animation; update every 50ms (20Hz)


Serial line: Matrix updated:
['2.2078', '2.0286', '2.2865']
['1.9196', '1.8089', '1.9703']
['2.1707', '2.1275', '2.2446']
['2.2320', '2.2905', '2.4017']
['2.1734', '2.2341', '2.3939']
Serial line: 
Serial line: 
Serial line: Matrix updated:
['2.2091', '2.0302', '2.2876']
['1.9209', '1.8097', '1.9684']
['2.1713', '2.1277', '2.2443']
['2.2336', '2.2897', '2.4012']
['2.1747', '2.2314', '2.3937']
Serial line: 
Serial line: 
Serial line: Matrix updated:
['2.2073', '2.0294', '2.2865']
['1.9209', '1.8089', '1.9709']
['2.1694', '2.1277', '2.2427']
['2.2336', '2.2919', '2.4033']
['2.1729', '2.2341', '2.3961']
Serial line: 
Text(0, 0, '2.21')
Text(1, 0, '2.03')
Text(2, 0, '2.29')
Text(0, 1, '1.92')
Text(1, 1, '1.81')
Text(2, 1, '1.97')
Text(0, 2, '2.17')
Text(1, 2, '2.13')
Text(2, 2, '2.25')
Text(0, 3, '2.23')
Text(1, 3, '2.29')
Text(2, 3, '2.41')
Text(0, 4, '2.17')
Text(1, 4, '2.23')
Text(2, 4, '2.40')
Serial line: 
Serial line: Matrix updated:
['2.2075', '2.0267', '2.2860']
['1.9212', '1.8100'

Serial line: Matrix updated:
['2.2083', '2.1474', '2.3085']
['1.9991', '1.9233', '2.0095']
['2.3061', '2.2755', '2.3451']
['2.5261', '2.5320', '2.5659']
['1.9816', '2.0797', '2.1576']
Serial line: 
Serial line: 
Serial line: Matrix updated:
['2.2097', '2.1487', '2.3123']
['2.0020', '1.9247', '2.0133']
['2.3080', '2.2779', '2.3488']
['2.5202', '2.5293', '2.5667']
['1.9830', '2.0821', '2.1629']
Serial line: 
Serial line: 
Serial line: Matrix updated:
['2.2097', '2.1479', '2.3083']
['2.0007', '1.9239', '2.0112']
['2.3069', '2.2752', '2.3440']
['2.5242', '2.5293', '2.5669']
['1.9838', '2.0858', '2.1619']
Serial line: 
Serial line: 
Serial line: Matrix updated:
['2.2064', '2.1479', '2.3074']
['2.0044', '1.9274', '2.0114']
['2.3048', '2.2715', '2.3426']
['2.5178', '2.5266', '2.5659']
['1.9757', '2.0775', '2.1551']
Serial line: 
Serial line: 
Serial line: Matrix updated:
['2.2067', '2.1482', '2.3083']
['1.9983', '1.9220', '2.0098']
['2.3007', '2.2682', '2.3375']
['2.5151', '2.5229', '2.5616']